# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,usogorsk,63.4106,48.6872,-28.52,100,98,0.47,RU,1707252176
1,1,puerto natales,-51.7236,-72.4875,18.25,42,100,4.12,CL,1707252130
2,2,kuqa,41.7278,82.9364,-3.93,71,38,1.53,CN,1707252445
3,3,udachny,66.4167,112.4000,-29.20,82,95,1.89,RU,1707252158
4,4,uturoa,-16.7333,-151.4333,28.06,85,100,12.13,PF,1707252140


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color = 'City',
    alpha = 0.5
    )
    
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df= city_data_df.loc[(city_data_df['Max Temp']>=21.03) & (city_data_df['Max Temp']<=26.27)]
ideal_humidity_df= ideal_temp_df.loc[(ideal_temp_df['Humidity']>=55) & (ideal_temp_df['Humidity']<=100)]
idea_weather_df = ideal_humidity_df.loc[(ideal_humidity_df['Wind Speed']>=0.35) & (ideal_humidity_df['Wind Speed']<=3.69)]


# Drop any rows with null values
ideal_temp_df.dropna(how='any', axis=0)

# Display sample data
idea_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,hawaiian paradise park,19.5933,-154.9731,23.91,77,100,2.57,US,1707252449
30,30,adamstown,-25.0660,-130.1015,24.98,80,31,1.70,PN,1707252128
37,37,visakhapatnam,17.6900,83.2093,23.18,82,0,2.10,IN,1707252180
42,42,nova sintra,14.8667,-24.7167,21.20,72,0,3.39,CV,1707252148
46,46,mananjary,-21.2167,48.3333,24.08,95,100,3.04,MG,1707252452
...,...,...,...,...,...,...,...,...,...,...
503,503,warangal,18.0000,79.5833,23.04,69,0,3.41,IN,1707252526
515,515,inongo,-1.9500,18.2667,25.51,70,38,1.57,CD,1707252527
521,521,waingapu,-9.6567,120.2641,25.34,83,79,2.44,ID,1707252528
523,523,posse,-14.0931,-46.3694,25.91,67,100,1.50,BR,1707252528


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idea_weather_df.drop(columns=['City_ID','Cloudiness','Wind Speed','Max Temp','Date']) 
hotel_df=hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,'Hotel Name','')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,hawaiian paradise park,US,19.5933,-154.9731,77,
30,adamstown,PN,-25.0660,-130.1015,80,
37,visakhapatnam,IN,17.6900,83.2093,82,
42,nova sintra,CV,14.8667,-24.7167,72,
46,mananjary,MG,-21.2167,48.3333,95,
...,...,...,...,...,...,...
503,warangal,IN,18.0000,79.5833,69,
515,inongo,CD,-1.9500,18.2667,70,
521,waingapu,ID,-9.6567,120.2641,83,
523,posse,BR,-14.0931,-46.3694,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,'Lat']
    longitude = hotel_df.at[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = f'{base_url}?categories=accommodation.hotel&filter={params["filter"]}&bias={params["bias"]}&apiKey={geoapify_key}'
    
    # Convert the API response to JSON format
    name_address = requests.get(name_address).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
visakhapatnam - nearest hotel: Fairfield by Marriot
nova sintra - nearest hotel: Residência Ka Dencho
mananjary - nearest hotel: Ideal Hotel Annexe
gamboma - nearest hotel: No hotel found
kencong - nearest hotel: Grand Gumuk Mas Hotel
tsiombe - nearest hotel: No hotel found
okakarara - nearest hotel: No hotel found
manado - nearest hotel: Swiss Belhotel
kroya - nearest hotel: Setia Hotel
richards bay - nearest hotel: SUN1 Richards Bay
saint-pierre - nearest hotel: Tropic Hotel
tshela - nearest hotel: No hotel found
flying fish cove - nearest hotel: Christmas Island Lodge
pangkalanbuun - nearest hotel: Hotel Tiara
maroantsetra - nearest hotel: Coco Beach
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
kutoarjo - nearest hotel: Hotel Kencana
hilo - nearest hotel: Dolphin Bay Hotel
voi - nearest hotel: Fine Breeze Hotel
bamboo flat - nearest hotel: Hotel Mega

,City,Country,Lat,Lng,Humidity,Hotel Name
24,hawaiian paradise park,US,19.5933,-154.9731,77,No hotel found
30,adamstown,PN,-25.0660,-130.1015,80,No hotel found
37,visakhapatnam,IN,17.6900,83.2093,82,Fairfield by Marriot
42,nova sintra,CV,14.8667,-24.7167,72,Residência Ka Dencho
46,mananjary,MG,-21.2167,48.3333,95,Ideal Hotel Annexe
...,...,...,...,...,...,...
503,warangal,IN,18.0000,79.5833,69,city grand
515,inongo,CD,-1.9500,18.2667,70,Chez LELE IYOLO
521,waingapu,ID,-9.6567,120.2641,83,HOTEL SANDLEWOOD
523,posse,BR,-14.0931,-46.3694,67,Palmeiras Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = 'City',
    hover_cols=['Hotel Name','Country'],
    alpha = 0.5,
    size = 'Humidity'
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)